# Demo Stock Visualization

## Installation of packages

In [3]:
# !pip install taipy
# !pip install yfinance
# !pip install prophet

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached lxml-4.9.1-cp310-cp310-win_amd64.whl (3.6 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached prophet-1.1.1-cp310-cp310-win_amd64.whl (12.1 MB)
  Using cached holidays-0.16-py3-none-any.whl (184 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
     -------------------------------------- 81.2/81.2 kB 909.3 kB/s eta 0:00:00
  Using cached matplotlib-3.6.1-cp310-cp310-win_amd64.whl (7.2 MB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached PyMeeus-0.5.11-py3-none-any.whl
  Using cached hijri_converter-2.2.4-py3-none-any.whl (14 kB)
  Using cached korean_lunar_cale

## Import of packages

In [4]:
from taipy import Gui 
from datetime import date

import yfinance as yf
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


## Initialization

In [5]:
start_date = "2015-01-01"
end_date = date.today().strftime("%Y-%m-%d")

selected_stock = "AAPL"

n_years = 1
data = None

show_part_1 = True
show_part_2 = True
show_dialog = False

## Functions

In [7]:
def load_data(ticker, start, end):
    data = yf.download(ticker, start, end) #downloading the stock data from START to TODAY
    data.reset_index(inplace=True) #put date in the first column
    data['Date'] = data['Date'].dt.tz_localize(None)
    return data

def generate_hist_data(state):
    print("GENERATING HIST DATA")
    print(state.start_date)
    state.data = load_data(state.selected_stock, state.start_date, state.end_date)
    state.show_part_1 = True

def generate_forecast_data(data, n_years):
    print("FORECASTING")
    #FORECASTING
    df_train = data[['Date', 'Close']]
    df_train = df_train.rename(columns={"Date": "ds","Close": "y"}) #This is the format that Prophet accepts

    m = Prophet()
    m.fit(df_train)
    future = m.make_future_dataframe(periods=n_years*365)
    forecast = m.predict(future)[['ds','yhat_lower','yhat_upper']]
    print("Process Completed!")

    return forecast


def forecast_display(state):
    state.forecast = generate_forecast_data(state.data, state.n_years)

In [8]:
data = load_data(selected_stock, start_date, end_date)
forecast = generate_forecast_data(data, n_years)

[*********************100%***********************]  1 of 1 completed
FORECASTING


12:55:03 - cmdstanpy - INFO - Chain [1] start processing
12:55:04 - cmdstanpy - INFO - Chain [1] done processing


Process Completed!


## Markdown for pages

In [10]:
page = """
<center><h1>Stock Price Analysis Dashboard with Taipy</h1></center>

<|layout|columns=1 2 2|

<|
##Choose the period
###FROM:   
<|{start_date}|date|>  
###TO:   
<|{end_date}|date|>  
|>

<|
##Please enter a valid ticker:  
<center>
<|{selected_stock}|input|>  
</center>
##Popular tickers:
<center>
<|{selected_stock}|toggle|lov=MSFT;GOOG;AAPL; AMZN; META; COIN; GME; AMC;PYPL|>
</center>
|>





<|
##Select a prediction method
Select number of prediction years: <|{n_years}|>  
<|{n_years}|slider|min=1|max=5|>  
|>
|>


<|ENTER|button|on_action=generate_hist_data|>  
<|Show dialog|button|on_action={lambda state: state.assign("show_dialog", True)}|>




<|layout|columns=1 1|
<|
<|part|render={show_part_1}|partial={partial_A}|>

|>

<|
<|PREDICT|button|on_action=forecast_display|>
<|part|render={show_part_2}|partial={partial_B}|>
|>
|>


<|Historical Data|expandable|expanded=False|partial={partial_A}|>
<|{show_dialog}|dialog|width=80%|partial={partial_A}|on_action={lambda state: state.assign("show_dialog", False)}|>
<|FORECAST Data|expandable|expanded=False|partial={partial_B}|>
"""

In [11]:
p1 = """
##Historical closing price
<|{data}|chart|mode=line|x=Date|y[1]=Open|y[2]=Close|>

##Historical daily trading volume
<|{data}|chart|mode=line|x=Date|y=Volume|>

##Here is the historical data of the stock: <|{selected_stock}|>
<|{data}|table|width=80%|>
"""

In [12]:
p2 = """
##FORECAST DATA 
<|{forecast}|chart|mode=line|x=ds|y[1]=yhat_lower|y[2]=yhat_upper|>

<|{forecast}|table|width=80%|>
"""

## Gui creation

In [14]:
#Run Taipy GUI
gui = Gui(page)
partial_A = gui.add_partial(p1)
partial_B = gui.add_partial(p2)

gui.run(dark_mode=False, port=5001)

 * Server starting on http://127.0.0.1:5001
